We are uploading all csv files of the Scarlet & Violet main and special sets.

Sets: SV01, SV02, SV03, SV: Scarlet & Violet 151, SV04, SV: Paldean Fates, SV05, SV06, SV: Shrouded Fable, SV07, SV08, SV: Prismatic Evolutions, SV09

In [47]:
import pandas as pd
import scipy as sp

sv01_df = pd.read_csv('SV01ScarletAndVioletBaseSetProductsAndPrices.csv')
sv02_df = pd.read_csv('SV02PaldeaEvolvedProductsAndPrices.csv')
sv03_df = pd.read_csv('SV03ObsidianFlamesProductsAndPrices.csv')
sv04_df = pd.read_csv('SV04ParadoxRiftProductsAndPrices.csv')
sv05_df = pd.read_csv('SV05TemporalForcesProductsAndPrices.csv')
sv06_df = pd.read_csv('SV06TwilightMasqueradeProductsAndPrices.csv')
sv07_df = pd.read_csv('SV07StellarCrownProductsAndPrices.csv')
sv08_df = pd.read_csv('SV08SurgingSparksProductsAndPrices.csv')
sv09_df = pd.read_csv('SV09JourneyTogetherProductsAndPrices.csv')
sv_151_df = pd.read_csv('SVScarletAndViolet151ProductsAndPrices.csv')
sv_pf_df = pd.read_csv('SVPaldeanFatesProductsAndPrices.csv')
sv_sf_df = pd.read_csv('SVShroudedFableProductsAndPrices.csv')
sv_pe_df = pd.read_csv('SVPrismaticEvolutionsProductsAndPrices.csv')

We then merge all of these sets into a master set, filter out unnecessary columns, and add a column for the textual set name based on groupId variable.

In [48]:
master_df = pd.concat([sv01_df, sv02_df, sv03_df, sv04_df, sv05_df, sv06_df, sv07_df, sv08_df, sv09_df, sv_151_df, sv_pf_df, sv_sf_df, sv_pe_df])
master_df = master_df[['productId', 'name', 'groupId', 'lowPrice', 'highPrice', 'marketPrice', 'extRarity']]

# filter for cards of specific rarity
master_df = master_df[(master_df['extRarity'] == 'Special Illustration Rare') | (master_df['extRarity'] == 'Illustration Rare') | (master_df['extRarity'] == 'Hyper Rare')]

# add textual set name column
def get_textset(setId):
    setId_list = [22873, 23120, 23228, 23286, 23381, 23473, 23537, 23651, 24073, 23237, 23353, 23529, 23821]
    textId_list = [
        'Scarlet And Violet Base',
        'Paldea Evolved',
        'Obsidian Flames',
        'Paradox Rift',
        'Temporal Forces',
        'Twilight Masquerade',
        'Stellar Crown',
        'Surging Sparks',
        'Journey Together',
        'Scarlet And Violet 151',
        'Paldean Fates',
        'Shrouded Fable',
        'Prismatic Evolutions'
    ]
    index = setId_list.index(setId)
    return textId_list[index]

master_df['set'] = master_df['groupId'].apply(get_textset)

# add release date
def get_dateset(setId):
    name_list = ['Scarlet And Violet Base',
        'Paldea Evolved',
        'Obsidian Flames',
        'Paradox Rift',
        'Temporal Forces',
        'Twilight Masquerade',
        'Stellar Crown',
        'Surging Sparks',
        'Journey Together',
        'Scarlet And Violet 151',
        'Paldean Fates',
        'Shrouded Fable',
        'Prismatic Evolutions']
    date_list = [
        '05-31-2023',
        '06-09-2023',
        '08-11-2023',
        '10-03-2023',
        '03-22-2024',
        '05-24-2024',
        '09-13-2024',
        '11-08-2024',
        '03-28-2025',
        '09-22-2023',
        '01-26-2024',
        '09-02-2024',
        '01-17-2025'
    ]
    index = name_list.index(setId)
    return date_list[index]

master_df['set'] = master_df['groupId'].apply(get_textset)
master_df['release date'] = master_df['set'].apply(get_dateset)

# sort dataframe by card rarity
master_df = master_df.sort_values(by=['groupId', 'extRarity'])

# reset index
master_df = master_df.reset_index()
master_df.index +=1

master_df['release date'] = pd.to_datetime(master_df['release date'], format='%m-%d-%Y')

display(master_df)

We will also have to fill in all NaN values, especially for the market prices of recent sets (ie. SV09 Journey Together).

In order to fill in the NaN values, we will take the average of the lowPrice and highPrice values and input the average into marketPrice.

In [49]:
def fill_market_price(row):
    if pd.isna(row['marketPrice']):
        low_high_average = (row['lowPrice'] + row['highPrice'])/2
        row['marketPrice'] = round(low_high_average, 2)
    return row['marketPrice']

master_df['marketPrice'] = master_df.apply(fill_market_price, axis=1)

We will now conduct three hypothesis tests to observe trends and patterns in data.

TEST I: Using an ANOVA test, we plan on determining if the average price of cards of the same rarity vary significantly across all relevant sets
(relevant sets - scarlet and violet main and special sets)

Because we are testing multiple categories and observing if they exist within the same distribution, an ANOVA test would be appropriate.
Significance level = 0.05 (Level of confidence = 95%)

H0: There is no significant difference in the prices of the same card rarity across the relevant sets.
HA: There exists significant differences in the prices of the same card rarity across the relevant sets.

In [50]:
hr_df = master_df[master_df['extRarity'] == 'Hyper Rare']
ir_df = master_df[master_df['extRarity'] == 'Illustration Rare']
sir_df = master_df[master_df['extRarity'] == 'Special Illustration Rare']

average_prices = pd.DataFrame(columns=['set', 'averagePrice'])

Hyper Rare Cards:

In [51]:
# for Hyper Rare cards
hr_sets = hr_df.groupby('set', sort=False)

for set,group in hr_sets:
    group = group.dropna(subset=['marketPrice']).sort_values(by='marketPrice')
    rarity_mean = group['marketPrice'].mean()
    row = {'set': set, 'averagePrice': round(rarity_mean, 2)}
    average_prices = pd.concat([average_prices, pd.DataFrame([row])], ignore_index=True)

hr_avg_prices = [group['marketPrice'].dropna().values for _, group in hr_df.groupby('set', sort=False)]

hr_res = sp.stats.f_oneway(*hr_avg_prices)
hr_tukey = sp.stats.tukey_hsd(*hr_avg_prices)
print(hr_res.pvalue)
print(hr_tukey)

We will use pairwise T-tests to determine if there exists a significant difference between the volatility in prices between rarities. Here, we define volatility as the difference between a cards high and low price points. We'll have a significance level of $\alpha = 0.5$. 

$H_0:$ There is no significant difference in the volatility in prices between rarities. 

$H_a:$ There is significant difference in the volatility in prices between rarities. 

In [53]:
from scipy.stats import ttest_ind

ttest_hr_df = master_df[master_df['extRarity'] == 'Hyper Rare']
ttest_ir_df = master_df[master_df['extRarity'] == 'Illustration Rare']
ttest_sir_df = master_df[master_df['extRarity'] == 'Special Illustration Rare']

# df.apply(lambda row: row['Val10']-row['Val1'])
#ttest_hr_df['volatility'] = ttest_hr_df.apply(lambda x: x['highPrice'] - x ['lowPrice'])
# [[ttest_hr_df['highPrice']]-ttest_hr_df['lowPrice']]
ttest_ir_df['volatility'] = ttest_ir_df[ttest_ir_df['highPrice']-ttest_ir_df['lowPrice']]
ttest_sir_df['volatility'] = ttest_sir_df[ttest_sir_df['highPrice']-ttest_sir_df['lowPrice']]


KeyError: "None of [Index([             71.01,               40.7,               52.5,\n       26.009999999999998, 14.049999999999999,              14.93,\n                     48.0,               80.0,              23.45,\n                    270.0,\n       ...\n                    21.97,  7.350000000000001,              16.01,\n        7.489999999999998, 19.040000000000003, 45.099999999999994,\n                    23.55, 6.0600000000000005, 3.9899999999999984,\n        9.990000000000002],\n      dtype='float64', length=231)] are in the [columns]"